<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

#  Mind Utils Generation

Many news recommendation methods ultilize word embeddings, news vertical embeddings, news subvertical embeddings and user id embedding. Therefore, it is necessary to generate a word dictionary, a vertical dictionary, a subvertical dictionary and a userid dictionary to convert words, news verticals, subvericals and user ids from strings to indexes. To use the pretrain word embedding, a embedding matrix is generated as the intial weight of the word embedding layer.


This notebook gives examples about how to generate
* word_dict.pkl: convert the words in news titles into indexes.
* word_dict_all.pkl: convert the words in news titles and abstracts into indexes.
* embedding.npy: pretrained word embedding matrix of words in word_dict.pkl
* embedding_all.npy: pretrained embedding matrix of words in word_dict_all.pkl
* vert_dict.pkl: convert news verticals into indexes.
* subvert_dict.pkl: convert news subverticals into indexes.
* uid2index.pkl: convert user ids into indexes.

In [1]:
import sys
sys.path.append("../../")
import os
import pandas as pd
from collections import Counter
from tqdm import tqdm
import pickle
import numpy as np
import scrapbook as sb

from tempfile import TemporaryDirectory
from reco_utils.dataset.mind import (download_mind,
                                     extract_mind,
                                     download_and_extract_globe,
                                     load_glove_matrix,
                                     word_tokenize
                                    )
from reco_utils.dataset.download_utils import unzip_file

print("System version: {}".format(sys.version))


System version: 3.6.11 | packaged by conda-forge | (default, Nov 27 2020, 18:51:43) 
[GCC Clang 11.0.0]


In [2]:
mind_type="demo"
# word_embedding_dim should be in [50, 100, 200, 300]
word_embedding_dim = 300

In [3]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name
train_zip, valid_zip = download_mind(size=mind_type, dest_path=data_path)
unzip_file(train_zip, os.path.join(data_path, 'train'), clean_zip_file=False)
unzip_file(valid_zip, os.path.join(data_path, 'valid'), clean_zip_file=False)
output_path = os.path.join(data_path, 'utils')
os.makedirs(output_path, exist_ok=True)

100%|██████████| 17.0k/17.0k [00:42<00:00, 403KB/s]
100%|██████████| 9.84k/9.84k [00:14<00:00, 694KB/s]  


## Prepare utils of news

* word dictionary
* vertical dictionary
* subvetical dictionary

In [4]:
news = pd.read_table(os.path.join(data_path, 'train', 'news.tsv'),
                     names=['newid', 'vertical', 'subvertical', 'title',
                            'abstract', 'url', 'entities in title', 'entities in abstract'],
                     usecols = ['vertical', 'subvertical', 'title', 'abstract'])

In [5]:
news.head()

,vertical,subvertical,title,abstract
0,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
2,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
3,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."
4,weather,weathertopstories,It's been Orlando's hottest October ever so fa...,There won't be a chill down to your bones this...


In [6]:
news_vertical = news.vertical.drop_duplicates().reset_index(drop=True)
vert_dict_inv = news_vertical.to_dict()
vert_dict = {v: k+1 for k, v in vert_dict_inv.items()}

news_subvertical = news.subvertical.drop_duplicates().reset_index(drop=True)
subvert_dict_inv = news_subvertical.to_dict()
subvert_dict = {v: k+1 for k, v in vert_dict_inv.items()}

In [7]:
news.title = news.title.apply(word_tokenize)
news.abstract = news.abstract.apply(word_tokenize)

In [8]:
word_cnt = Counter()
word_cnt_all = Counter()

for i in tqdm(range(len(news))):
    word_cnt.update(news.loc[i]['title'])
    word_cnt_all.update(news.loc[i]['title'])
    word_cnt_all.update(news.loc[i]['abstract'])

100%|██████████| 26740/26740 [00:11<00:00, 2257.38it/s]


In [9]:
word_dict = {k: v+1 for k, v in zip(word_cnt, range(len(word_cnt)))}
word_dict_all = {k: v+1 for k, v in zip(word_cnt_all, range(len(word_cnt_all)))}

In [10]:
with open(os.path.join(output_path, 'vert_dict.pkl'), 'wb') as f:
    pickle.dump(vert_dict, f)
    
with open(os.path.join(output_path, 'subvert_dict.pkl'), 'wb') as f:
    pickle.dump(subvert_dict, f)

with open(os.path.join(output_path, 'word_dict.pkl'), 'wb') as f:
    pickle.dump(word_dict, f)
    
with open(os.path.join(output_path, 'word_dict_all.pkl'), 'wb') as f:
    pickle.dump(word_dict, f)

## Prepare embedding matrixs
* embedding.npy
* embedding_all.npy

In [11]:
glove_path = download_and_extract_globe(data_path)

100%|██████████| 842k/842k [06:30<00:00, 2.15kKB/s] 


In [12]:
embedding_matrix, exist_word = load_glove_matrix(glove_path, word_dict, word_embedding_dim)
embedding_all_matrix, exist_all_word = load_glove_matrix(glove_path, word_dict_all, word_embedding_dim)

400000it [00:08, 47154.93it/s]
400000it [00:09, 43258.03it/s]


In [13]:
np.save(os.path.join(output_path, 'embedding.npy'), embedding_matrix)
np.save(os.path.join(output_path, 'embedding_all.npy'), embedding_all_matrix)

## Prepare uid2index.pkl

In [14]:
uid2index = {}

with open(os.path.join(data_path, 'train', 'behaviors.tsv'), 'r') as f:
    for l in tqdm(f):
        uid = l.strip('\n').split('\t')[1]
        if uid not in uid2index:
            uid2index[uid] = len(uid2index) + 1

22034it [00:00, 370992.88it/s]


In [15]:
with open(os.path.join(output_path, 'uid2index.pkl'), 'wb') as f:
    pickle.dump(uid2index, f)

In [16]:
utils_state = {
    'vert_num': len(vert_dict),
    'subvert_num': len(subvert_dict),
    'word_num': len(word_dict),
    'word_num_all': len(word_dict_all),
    'embedding_exist_num': len(exist_word),
    'embedding_exist_num_all': len(exist_all_word),
    'uid2index': len(uid2index)
}

In [ ]:
sb.glue("utils_state", utils_state)

In [ ]:
tmpdir.cleanup()